In [1]:
import re
import pandas as pd
#from datasets import load_dataset
import time
import pickle
from tqdm import tqdm

In [9]:
df1=pd.read_pickle("all_data_finals/atFinal-gpt_syn_multitool-1.pkl")
df2=pd.read_pickle("all_data_finals/atFinal-maths600.pkl")
df3=pd.read_pickle("all_data_finals/atFinal-wikitool.pkl")
df4=pd.read_csv("all_data_finals/final_common_sense_gpt_validated.csv")


In [60]:
df5=pd.read_csv("all_data_finals/common_sense_gemini_validated.csv")
solved_rows_df = df5[df5['is_final_in_solved'] == True]
solved_rows_df.shape
df4 = pd.concat([df4, solved_rows_df], axis=0)
df4.reset_index(drop=True, inplace=True)
df4=df4.rename(columns={'question': 'Q', 'answer': 'A'})
df4.to_pickle('all_data_finals/common_sense_train.pkl')

(90, 9)

In [67]:
# Choose 'Q' and 'C' columns from each DataFrame
df1_selected = df1[['Q', 'C']]
df2_selected = df2[['Q', 'C']]
df3_selected = df3[['Q', 'C']]
df4_selected = df4[['Q', 'C']]

# Concatenate selected columns into a new DataFrame
new_df = pd.concat([df1_selected, df2_selected, df3_selected, df4_selected], ignore_index=True)

# Shuffle the rows
new_df = new_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [68]:
new_df.shape

(1543, 2)

In [129]:
new_df.head()

,Q,C
0,Aubrey and Ava are running a km long race. Aub...,First search [number of meters in km -Wiki-> y...
1,The 1950 BRDC International Trophy included wh...,First search [1950 BRDC International Trophy e...
2,"Anthony spends 6 hours at work, 6 hours on oth...",First search [number of hours in a day -Wiki->...
3,Rick has 130 cards. He decided to only keep 15...,"After keeping 15 cards, Rick had [130 - 15 = y..."
4,Who wrote and directed the film in which Adam ...,"First, determine [Adam Beach character in ""Sui..."


In [102]:
new_df.to_pickle("all_data_finals/Training_data.pkl")

In [132]:
new_df=pd.read_pickle("all_data_finals/Training_data.pkl")

In [133]:
from datasets import Dataset

In [138]:
# Splitting the DataFrame into train and validation sets
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(new_df, test_size=0.15, random_state=42)

In [139]:
# Reset index and drop the old index column for train_df
train_df = train_df.reset_index(drop=True)

# Reset index and drop the old index column for valid_df
valid_df = valid_df.reset_index(drop=True)

In [140]:
# Display the shapes of the train and validation sets
print("Train set shape:", train_df.shape)
print("Validation set shape:", valid_df.shape)

Train set shape: (1311, 2)
Validation set shape: (232, 2)


In [142]:
train_df.to_pickle("all_data_finals/tool_train.pkl")
valid_df.to_pickle("all_data_finals/tool_val.pkl")

In [149]:
## MISTRAL TEMPLATE

### NEW DATA

In [7]:
df1=pd.read_pickle("all_data_finals/atFinal-gpt_syn_multitool-1.pkl")
df2=pd.read_pickle("all_data_finals/atFinal-maths600.pkl")
df3=pd.read_pickle("all_data_finals/atFinal-wikitool.pkl")
df4=pd.read_pickle("all_data_finals/common_sense_train.pkl")

In [14]:
t1="Math"
t2="Wiki and QA"
t3="Wiki, QA and Math"

In [15]:
df1['Tools'] =t3
df2['Tools'] =t1
df3['Tools'] =t2
df4['Tools'] =t3

In [20]:
# Choose 'Q' and 'C' columns from each DataFrame
df1_selected = df1[['Q', 'C','Tools']]
df2_selected = df2[['Q', 'C','Tools']]
df3_selected = df3[['Q', 'C','Tools']]
df4_selected = df4[['Q', 'C','Tools']]

# Concatenate selected columns into a new DataFrame
new_df = pd.concat([df1_selected, df2_selected, df3_selected, df4_selected], ignore_index=True)

# Shuffle the rows
new_df = new_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [21]:
new_df.shape

(1543, 3)

In [22]:
new_df.to_pickle("all_data_finals/tool_training_data.pkl")

In [23]:
# Splitting the DataFrame into train and validation sets
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(new_df, test_size=0.15, random_state=42)
# Reset index and drop the old index column for train_df
train_df = train_df.reset_index(drop=True)
# Reset index and drop the old index column for valid_df
valid_df = valid_df.reset_index(drop=True)
# Display the shapes of the train and validation sets
print("Train set shape:", train_df.shape)
print("Validation set shape:", valid_df.shape)

Train set shape: (1311, 3)
Validation set shape: (232, 3)


In [24]:
train_df.to_pickle("all_data_finals/tool2_train.pkl")
valid_df.to_pickle("all_data_finals/tool2_val.pkl")

In [2]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""### INSTRUCTION
Your task is to generate a chain of abstractions (C) for the given question (Q) using the available tools: Wiki, QA, and Mathematical. You can use a single tool or a combination of tools to derive the answer (C). Follow the rules and formats provided for each tool:                               
**Tools:** 
1. **Wiki Tool:** Retrieves relevant articles from Wikipedia. * **Format:** `[search query -Wiki-> search query output]` 
2. **QA Tool:** Extracts focused answers from Wikipedia articles. * **Format:** `[input context -QA(question)-> output]` 
3. **Math Tool:** Solves mathematical computations based on information returned from the QA tool. * **Format:** `[polynomial expression]` (e.g., `[y1 + 20 = y2]`)

See examples below on how to decide which tools to use and their usage to generate C.
### EXAMPLES
 
Example 1 : Only Math tool used                                            
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees will the grove workers plant today?
Tools: Math
C: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been [21 - 15 = y1]. The answer is y1.

Example 2 : Wiki tool and QA tool used                                            
Q: Fritz von Brodowski was killed during what global war that lasted from 1939 to 1945?
Tools: Wiki and QA
C: Find the [war in which Fritz von Brodowski was killed -Wiki-> y1]. Fritz von Brodowski was killed in [y1 -QA(Fritz von Brodowski was killed in which war?)-> y2]. The answer is y2.

Example 3 : Wiki tool ,QA tool and Math tool used                                           
Q: What would be the length of the track where the 2013 Liqui Moly Bathurst 12 Hour was staged, if it would have been 4km longer?
Tools: Wiki, QA and Math
C: First search for [Mount Panorama Circuit -Wiki-> y1]. Length of circuit is [y1 -QA(what is the length of Mount Panorama Circuit ?)-> y2]. Length after adding 4km will be [4 + y2 = y3]. The answer is y3.

Now Generate C for the following Q.  Respond in following format
Tool: <Tools that should be used to answer the Q>
C: <chain of abstractions for Q>
                                  
### QUESTION
Q: {prompt_q}
                                                                                               
### RESPONSE
Tools: {tools_used}
C: {prompt_tool}
""")


## GO SEPARATE

In [10]:
df1=pd.read_pickle("all_data_finals/atFinal-gpt_syn_multitool-1.pkl")
df2=pd.read_pickle("all_data_finals/atFinal-maths600.pkl")
df3=pd.read_pickle("all_data_finals/atFinal-wikitool.pkl")
df4=pd.read_pickle("all_data_finals/common_sense_train.pkl")

In [11]:
df1_selected = df1[['Q', 'C']]
df2_selected = df2[['Q', 'C']]
df3_selected = df3[['Q', 'C']]
df4_selected = df4[['Q', 'C']]

In [12]:
# Concatenate selected columns into a new DataFrame
new_df = pd.concat([df1_selected, df4_selected], ignore_index=True)
# Shuffle the rows
new_df = new_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [13]:
# Splitting the DataFrame into train and validation sets
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(new_df, test_size=0.15, random_state=42)
# Reset index and drop the old index column for train_df
train_df = train_df.reset_index(drop=True)
# Reset index and drop the old index column for valid_df
valid_df = valid_df.reset_index(drop=True)
# Display the shapes of the train and validation sets
print("Train set shape:", train_df.shape)
print("Validation set shape:", valid_df.shape)

train_df.to_pickle("sep_data/multi-tool_train.pkl")
valid_df.to_pickle("sep_data/multi-tool_val.pkl")

Train set shape: (358, 2)
Validation set shape: (64, 2)


In [15]:
# Splitting the DataFrame into train and validation sets
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(df3, test_size=0.15, random_state=42)
# Reset index and drop the old index column for train_df
train_df = train_df.reset_index(drop=True)
# Reset index and drop the old index column for valid_df
valid_df = valid_df.reset_index(drop=True)
# Display the shapes of the train and validation sets
print("Train set shape:", train_df.shape)
print("Validation set shape:", valid_df.shape)

train_df.to_pickle("sep_data/wiki_train.pkl")
valid_df.to_pickle("sep_data/wiki_val.pkl")

Train set shape: (442, 9)
Validation set shape: (79, 9)
